## Summing all the prime numbers below a given number

In [1]:
import sys
import numba

# Default values (for Jupyter)
number = 2_500_000
ncores = 1

try:
    number = int(sys.argv[1])
except:
    pass

try:
    ncores = int(sys.argv[2])
except:
    pass

numba.set_num_threads(ncores)


In [2]:
import time

# Simple code

def if_prime(x):
    if x <= 1:
        return 0
    elif x <= 3:
        return x
    elif x % 2 == 0 or x % 3 == 0:
        return 0
    i = 5
    while i**2 <= x:
        if x % i == 0 or x % (i + 2) == 0:
            return 0
        i += 6
    return x

def sum_primes(x):
    result = 0
    for i in range(x):
        result += if_prime(i)
    return result


In [3]:
if ncores == 1:
    print("\n========== Python sequential ==========")
    suma = 0
    N = 3

    start = time.time()
    for i in range(N):
        suma = sum(map(if_prime, range(number)))
    stop = time.time()

    tiempo = (stop - start) / N

    print("The prime sum below ", number, "is ", suma,
          " and the time taken is", tiempo)
    
    tiempo = %timeit -r 2 -o -q sum_primes(number)
    suma = sum_primes(number)
    print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)
else:
    print("Skipping Python sequential version (ncores > 1)")


========== Python sequential ==========
The prime sum below  2500000 is  219697708195  and the time taken is 4.678010702133179
The prime sum below  2500000 is  219697708195  and the time taken is 4.62 s ± 6.55 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


In [4]:
import time
from numba import njit

@njit
def if_prime_numba(x):
    if x <= 1:
        return 0
    elif x <= 3:
        return x
    elif x % 2 == 0 or x % 3 == 0:
        return 0
    i = 5
    while i * i <= x:
        if x % i == 0 or x % (i + 2) == 0:
            return 0
        i += 6
    return x

@njit
def sum_primes_numba(x):
    result = 0
    for i in range(x):
        result += if_prime_numba(i)
    return result


In [5]:
# Warm-up
sum_primes_numba(10)

if ncores == 1:
    print("\n========== Numba sequential ==========")
    suma = 0
    N = 3

    start = time.time()
    for i in range(N):
        suma = sum_primes_numba(number)
    stop = time.time()

    tiempo = (stop - start) / N

    print("The prime sum below ", number, "is ", suma,
          " and the time taken is", tiempo)

    tiempo = %timeit -r 2 -o -q sum_primes_numba(number)
    suma = sum_primes_numba(number)
    print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)
    
else:
    print("Skipping Numba sequential version (ncores > 1)")


========== Numba sequential ==========
The prime sum below  2500000 is  219697708195  and the time taken is 0.18420990308125815
The prime sum below  2500000 is  219697708195  and the time taken is 184 ms ± 299 μs per loop (mean ± std. dev. of 2 runs, 10 loops each)


In [6]:
import multiprocessing as mp
from numba import njit

@njit
def if_prime_numba(x):
    if x <= 1:
        return 0
    elif x <= 3:
        return x
    elif x % 2 == 0 or x % 3 == 0:
        return 0
    i = 5
    while i * i <= x:
        if x % i == 0 or x % (i + 2) == 0:
            return 0
        i += 6
    return x

def partial_sum(args):
    start, end = args
    s = 0
    for i in range(start, end):
        s += if_prime_numba(i)
    return s

def sum_primes_pool(number, nproc=ncores):
    pool = mp.Pool(processes=nproc)
    chunk = number // nproc
    ranges = [(i*chunk, (i+1)*chunk) for i in range(nproc)]
    results = pool.map(partial_sum, ranges)
    pool.close()
    pool.join()
    return sum(results)


In [7]:
print("\n========== Multiprocessing Pool ==========")
suma = 0
N = 3

start = time.time()
for i in range(N):
    suma = sum_primes_pool(number, nproc=ncores)
stop = time.time()

tiempo = (stop - start) / N

print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)


========== Multiprocessing Pool ==========
The prime sum below  2500000 is  219697708195  and the time taken is 0.6706704298655192


In [8]:
from numba import njit, prange
import numba
import time

@njit
def if_prime(x):
    if x <= 1:
        return 0
    elif x <= 3:
        return x
    elif x % 2 == 0 or x % 3 == 0:
        return 0
    i = 5
    while i * i <= x:
        if x % i == 0 or x % (i + 2) == 0:
            return 0
        i += 6
    return x

@njit(parallel=True)
def sum_primes_prange(x):
    result = 0
    for i in prange(x):
        result += if_prime(i)
    return result


In [9]:
# Warm-up
sum_primes_prange(10)

print("\n========== Numba prange ==========")
suma = 0
N = 3

start = time.time()
for i in range(N):
    suma = sum_primes_prange(number)
stop = time.time()

tiempo = (stop - start) / N

print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)

tiempo = %timeit -r 2 -o -q sum_primes_prange(number)
suma = sum_primes_prange(number)

print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)



========== Numba prange ==========
The prime sum below  2500000 is  219697708195  and the time taken is 0.2041931947072347
The prime sum below  2500000 is  219697708195  and the time taken is 204 ms ± 229 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)


In [11]:
import numba
import time

# Force 4 threads (requested in the exercise)
numba.set_num_threads(4)

print("Numba threads:", numba.get_num_threads())

# Warm-up
sum_primes_prange(10)

print("\n========== Numba prange ==========")
suma = 0
N = 3

start = time.time()
for i in range(N):
    suma = sum_primes_prange(number)
stop = time.time()

tiempo = (stop - start) / N

print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)

tiempo = %timeit -r 2 -o -q sum_primes_prange(number)
suma = sum_primes_prange(number)

print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)


Numba threads: 4

========== Numba prange ==========
The prime sum below  2500000 is  219697708195  and the time taken is 0.07623044649759929
The prime sum below  2500000 is  219697708195  and the time taken is 70.6 ms ± 199 μs per loop (mean ± std. dev. of 2 runs, 10 loops each)


### Salida para 10^6:

Number = 1000000

CPUs   = 1


========== Python sequential ==========

The prime sum below  1000000 is  37550402023  and the time taken is 2.1713240146636963

The prime sum below  1000000 is  37550402023  and the time taken is 2.25 s ± 29.1 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


========== Numba sequential ==========

The prime sum below  1000000 is  37550402023  and the time taken is 0.10117729504903157

The prime sum below  1000000 is  37550402023  and the time taken is 101 ms ± 62.5 μs per loop (mean ± std. dev. of 2 runs, 10 loops each)

========== Multiprocessing Pool ==========

The prime sum below  1000000 is  37550402023  and the time taken is 0.489859660466512

========== Numba prange ==========

The prime sum below  1000000 is  37550402023  and the time taken is 0.11465644836425781

The prime sum below  1000000 is  37550402023  and the time taken is 115 ms ± 17.6 μs per loop (mean ± std. dev. of 2 runs, 10 loops each)

Number = 1000000

CPUs   = 2

Skipping Python sequential version (ncores > 1)

Skipping Numba sequential version (ncores > 1)

========== Multiprocessing Pool ==========

The prime sum below  1000000 is  37550402023  and the time taken is 0.3322571913401286

========== Numba prange ==========

The prime sum below  1000000 is  37550402023  and the time taken is 0.07148249944051106

The prime sum below  1000000 is  37550402023  and the time taken is 70.7 ms ± 21 μs per loop (mean ± std. dev. of 2 runs, 10 loops each)

Number = 1000000

CPUs   = 4

Skipping Python sequential version (ncores > 1)

Skipping Numba sequential version (ncores > 1)

========== Multiprocessing Pool ==========

The prime sum below  1000000 is  37550402023  and the time taken is 0.25482598940531415

========== Numba prange ==========

The prime sum below  1000000 is  37550402023  and the time taken is 0.03771233558654785

The prime sum below  1000000 is  37550402023  and the time taken is 37.7 ms ± 23.7 μs per loop (mean ± std. dev. of 2 runs, 10 loops each)

Number = 1000000

CPUs   = 8

Skipping Python sequential version (ncores > 1)

Skipping Numba sequential version (ncores > 1)

========== Multiprocessing Pool ==========

The prime sum below  1000000 is  37550402023  and the time taken is 0.2269911766052246

========== Numba prange ==========

The prime sum below  1000000 is  37550402023  and the time taken is 0.019401152928670246

The prime sum below  1000000 is  37550402023  and the time taken is 19.3 ms ± 2.14 μs per loop (mean ± std. dev. of 2 runs, 100 loops each)


### Salida para 10^7:

Number = 10000000

CPUs   = 1

========== Python sequential ==========

The prime sum below  10000000 is  3203324994356  and the time taken is 57.57837303479513

The prime sum below  10000000 is  3203324994356  and the time taken is 58.8 s ± 5.21 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)

========== Numba sequential ==========

The prime sum below  10000000 is  3203324994356  and the time taken is 2.5355921586354575

The prime sum below  10000000 is  3203324994356  and the time taken is 2.54 s ± 540 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)

========== Multiprocessing Pool ==========

The prime sum below  10000000 is  3203324994356  and the time taken is 5.199615875879924

========== Numba prange ==========

The prime sum below  10000000 is  3203324994356  and the time taken is 2.8675432999928794

The prime sum below  10000000 is  3203324994356  and the time taken is 2.87 s ± 85.7 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)

Number = 10000000

CPUs   = 2

Skipping Python sequential version (ncores > 1)

Skipping Numba sequential version (ncores > 1)

========== Multiprocessing Pool ==========

The prime sum below  10000000 is  3203324994356  and the time taken is 3.0124058723449707

========== Numba prange ==========

The prime sum below  10000000 is  3203324994356  and the time taken is 1.7896785736083984

The prime sum below  10000000 is  3203324994356  and the time taken is 1.79 s ± 285 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)

Number = 10000000

CPUs   = 4

Skipping Python sequential version (ncores > 1)

Skipping Numba sequential version (ncores > 1)

========== Multiprocessing Pool ==========

The prime sum below  10000000 is  3203324994356  and the time taken is 1.6574796040852864

========== Numba prange ==========

The prime sum below  10000000 is  3203324994356  and the time taken is 0.9580111503601074

The prime sum below  10000000 is  3203324994356  and the time taken is 957 ms ± 6.5 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)

Number = 10000000

CPUs   = 8

Skipping Python sequential version (ncores > 1)

Skipping Numba sequential version (ncores > 1)

========== Multiprocessing Pool ==========

The prime sum below  10000000 is  3203324994356  and the time taken is 0.9248718420664469

========== Numba prange ==========

The prime sum below  10000000 is  3203324994356  and the time taken is 0.4939850966135661

The prime sum below  10000000 is  3203324994356  and the time taken is 492 ms ± 3.12 μs per loop (mean ± std. dev. of 2 runs, 1 loop 
each)

### Python sequential

El bloque identificado como Python sequential corresponde al código original en Python, sin ningún tipo de optimización.
Para **10^6**, el tiempo de ejecución es de aproximadamente 2.2 s, mientras que para **10^7** el tiempo aumenta hasta ~58 s. Estos resultados muestran claramente que el código secuencial puro no escala adecuadamente y se vuelve impracticable para tamaños grandes del problema. Por este motivo, esta versión se ejecuta únicamente con un núcleo y se utiliza como referencia.

### Numba sequential

El bloque Numba sequential corresponde a la versión secuencial optimizada mediante el decorador @njit.
Gracias a la compilación JIT, el tiempo de ejecución se reduce a ~0.10 s para **10^6** y a ~2.5 s para **10^7** , lo que supone una aceleración superior a un orden de magnitud respecto al código original. Este resultado pone de manifiesto que Numba es una herramienta muy eficaz incluso sin paralelización.

### Multiprocessing Pool

El bloque Multiprocessing Pool muestra los resultados obtenidos al paralelizar el cálculo utilizando el módulo multiprocessing y un conjunto de procesos gestionados mediante Pool.
Para **10^6**, el tiempo disminuye desde ~0.49 s (1 núcleo) hasta ~0.23 s (8 núcleos), y para **10^7** se reduce desde ~5.2 s (1 núcleo) hasta ~0.92 s (8 núcleos). Aunque se observa una mejora clara al aumentar el número de núcleos, la escalabilidad no es lineal debido al overhead asociado a la creación de procesos y a la comunicación entre ellos.

### Numba prange

El bloque Numba prange corresponde a la implementación paralela con Numba, utilizando el decorador @njit(parallel=True) y la directiva prange.
Esta versión es la que presenta los mejores resultados. Para **10^6**, el tiempo de ejecución pasa de ~115 ms (1 núcleo) a ~19 ms (8 núcleos), mientras que para **10^7** se reduce de ~2.9 s (1 núcleo) a ~0.49 s (8 núcleos). La mejora se debe a que Numba gestiona internamente el paralelismo mediante hilos, evitando el overhead de la paralelización basada en procesos.